In [38]:
# ! pip install ai21
# ! pip install swarms

import os
from typing import List

from ai21 import AI21Client
from ai21.models.chat import ChatMessage
from dotenv import load_dotenv

from swarms import Agent, BaseLLM, MixtureOfAgents

load_dotenv()

class Jamba(BaseLLM):
    def __init__(self, api_key: str = os.getenv("AI21_API_KEY"), temperature: int = 0.8, max_tokens: int = 200):
        """
        Initializes the Jamba class with the provided API key.

        Args:
            api_key (str): The API key for the AI21Client.
        """
        os.environ["AI21_API_KEY"] = api_key
        self.api_key = api_key
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.client = AI21Client()

    def run(self, prompt: str, *args, **kwargs) -> str:
        """
        Generates a response for the given prompt using the AI21 model.

        Args:
            prompt (str): The prompt for generating the response.

        Returns:
            str: The generated response.
        
        Raises:
            Exception: If there is an issue with the API request.
        """
        try:
            response = self.client.chat.completions.create(
                model="jamba-instruct-preview",  # Latest model
                messages=[ChatMessage(
                    role="user",
                    content=prompt
                )],
                temperature=self.temperature,
                max_tokens=self.max_tokens,
                *args, 
                **kwargs
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error: {e}")
            raise

# Example usage:
# jamba = Jamba()
# out = jamba.run("Write a product title for a sports T-shirt to be published on an")
# print(out)


model = Jamba(
    max_tokens=4000,
)

# Agent Planner
This agent below will make the plan for the agent.

In [39]:

BOSS_PLANNER = """
You're the swarm orchestrator agent

**Objective:** Your task is to intake a business problem or activity and create a swarm of specialized LLM agents that can efficiently solve or automate the given problem. You will define the number of agents, specify the tools each agent needs, and describe how they need to work together, including the communication protocols.

**Instructions:**

1. **Intake Business Problem:**
   - Receive a detailed description of the business problem or activity to automate.
   - Clarify the objectives, constraints, and expected outcomes of the problem.
   - Identify key components and sub-tasks within the problem.

2. **Agent Design:**
   - Based on the problem, determine the number and types of specialized LLM agents required.
   - For each agent, specify:
     - The specific task or role it will perform.
     - The tools and resources it needs to perform its task.
     - Any prerequisite knowledge or data it must have access to.
   - Ensure that the collective capabilities of the agents cover all aspects of the problem.

3. **Coordination and Communication:**
   - Define how the agents will communicate and coordinate with each other.
   - Choose the type of communication (e.g., synchronous, asynchronous, broadcast, direct messaging).
   - Describe the protocol for information sharing, conflict resolution, and task handoff.

4. **Workflow Design:**
   - Outline the workflow or sequence of actions the agents will follow.
   - Define the input and output for each agent.
   - Specify the triggers and conditions for transitions between agents or tasks.
   - Ensure there are feedback loops and monitoring mechanisms to track progress and performance.

5. **Scalability and Flexibility:**
   - Design the system to be scalable, allowing for the addition or removal of agents as needed.
   - Ensure flexibility to handle dynamic changes in the problem or environment.

6. **Output Specification:**
   - Provide a detailed plan including:
     - The number of agents and their specific roles.
     - The tools and resources each agent will use.
     - The communication and coordination strategy.
     - The workflow and sequence of actions.
   - Include a diagram or flowchart if necessary to visualize the system.

**Example Structure:**

**Business Problem:** Automate customer support for an e-commerce platform.

**Agents and Roles:**
1. **Customer Query Classifier Agent:**
   - Task: Classify incoming customer queries into predefined categories.
   - Tools: Natural language processing toolkit, pre-trained classification model.
   - Communication: Receives raw queries, sends classified queries to relevant agents.

2. **Order Status Agent:**
   - Task: Provide order status updates to customers.
   - Tools: Access to order database, query processing toolkit.
   - Communication: Receives classified queries about order status, responds with relevant information.

3. **Product Recommendation Agent:**
   - Task: Suggest products to customers based on their query and browsing history.
   - Tools: Recommendation engine, access to product database.
   - Communication: Receives classified queries about product recommendations, sends personalized suggestions.

4. **Technical Support Agent:**
   - Task: Assist customers with technical issues.
   - Tools: Access to technical support database, troubleshooting toolkit.
   - Communication: Receives classified queries about technical issues, provides solutions or escalation.

**Communication Strategy:**
- **Type:** Asynchronous communication through a central message broker.
- **Protocol:** Agents publish and subscribe to specific topics related to their tasks. 
- **Conflict Resolution:** If multiple agents need to handle the same query, a priority protocol is in place to determine the primary responder.

**Workflow:**
1. Customer Query Classifier Agent receives and classifies the query.
2. Classified query is routed to the appropriate specialized agent.
3. Specialized agent processes the query and sends a response.
4. If needed, the response triggers further actions from other agents.

**Scalability and Flexibility:**
- Agents can be added or removed based on query volume and complexity.
- System adapts to changes in query types and business needs.

**Output Plan:**
- Diagram illustrating agent roles and communication flow.
- Detailed description of each agent's tasks, tools, and communication methods.
- Workflow sequence from query intake to resolution.


"""


# Initialize the agent
planning_agent = Agent(
   agent_name="Boss Director",
   system_prompt=BOSS_PLANNER,
   agent_description="Generates a spec of agents for the problem at hand.",
   llm=model,
   max_loops=1,
   autosave=True,
   dynamic_temperature_enabled=True,
   dashboard=False,
   verbose=True,
   streaming_on=True,
   # interactive=True, # Set to False to disable interactive mode
   saved_state_path="accounting_agent.json",
   # tools=[calculate_profit, generate_report],
   # docs_folder="docs",
   # pdf_path="docs/accounting_agent.pdf",
   # tools=[browser_automation],
)

# out = planning_agent.run("Create a swarm of agents for automating customer support for an e-commerce platform.")


Initializing Autonomous Agent Boss Director...


# Agent creator
This agent is going to create the code for every agent

In [44]:
from swarms.tools.py_func_to_openai_func_str import get_openai_function_schema_from_func

# Name, system prompt, 
def create_and_execute_swarm(name: List[str], system_prompt: List[str], task: str):
    """
    Creates and executes a swarm of agents for the given task.

    Args:
        name (List[str]): A list of names for the agents.
        system_prompt (List[str]): A list of system prompts for the agents.
        task (str): The description of the task for the swarm.
        *args: Variable length argument list.
        **kwargs: Arbitrary keyword arguments.

    Returns:
        List[Agent]: A list of agents in the swarm.

    """
    agents = []
    for name, prompt in zip(name, system_prompt):
        agent = Agent(
            agent_name=name,
            system_prompt=prompt,
            agent_description="Generates a spec of agents for the problem at hand.",
            llm=model,
            max_loops=1,
            autosave=True,
            dynamic_temperature_enabled=True,
            dashboard=False,
            verbose=True,
            streaming_on=True,
            # interactive=True, # Set to False to disable interactive mode
            saved_state_path=f"{name}_agent.json",
            # tools=[calculate_profit, generate_report],
            # docs_folder="docs",
            # pdf_path="docs/accounting_agent.pdf",
            # tools=[browser_automation],
        )
        agents.append(agent)
        
    # MoA
    moa = MixtureOfAgents(agents=agents, description=task, final_agent=name[0])
    
    out = moa.run(task,)
    print(out)
    return out

function_schema = get_openai_function_schema_from_func(function=create_and_execute_swarm, name = create_and_execute_swarm.__name__, description=create_and_execute_swarm.__doc__)
# print(f"Function schema: {function_schema}")

BOSS_CREATOR = f"""

You are a swarm orchestrator with expertise in agentic design. Your task is to solve a business problem by creating and coordinating specialized LLM agents. Follow the schematic schema with function calling to design the solution.

Create a cohesive system of specialized LLM agents that effectively solve or automate the given business problem through clear roles, efficient communication, and a well-defined workflow. Ensure the system is scalable and flexible to adapt to changes.

Follow the following schema using markdown format and output only this and nothing else:
```json
{function_schema}
```

"""


# Boss Agent creator
boss_agent_creator = Agent(
    agent_name="Boss Agent Creator",
    system_prompt=BOSS_CREATOR,
    agent_description="Generates a spec of agents for the problem at hand.",
    llm=model,
    max_loops=1,
    autosave=True,
    dynamic_temperature_enabled=True,
    dashboard=False,
    verbose=True,
    streaming_on=True,
    # interactive=True, # Set to False to disable interactive mode
    saved_state_path="boss_director_agent.json",
    # tools=[calculate_profit, generate_report],
    # docs_folder="docs",
    # pdf_path="docs/accounting_agent.pdf",
    tools=[create_and_execute_swarm],
)

print("Agent generating function schema")
boss_agent_creator.run("Create a swarm of agents for automating customer support for an e-commerce platform.")

2024-06-22T17:17:18.845351-0700 Tools provided make sure the functions have documentation ++ type hints, otherwise tool execution won't be reliable.
2024-06-22T17:17:18.848651-0700 Tools provided: Accessing 1 tools
2024-06-22T17:17:18.849536-0700 Converting tools into OpenAI function calling schema
2024-06-22T17:17:18.850261-0700 Converting tool: create_and_execute_swarm into a OpenAI certified function calling schema. Add documentation and type hints.
2024-06-22T17:17:18.852247-0700 Conversion process successful, the tool create_and_execute_swarm has been integrated with the agent successfully.
2024-06-22T17:17:18.857608-0700 Autonomous Agent Activated.
2024-06-22T17:17:18.858788-0700 All systems operational. Executing task...
2024-06-22T17:17:18.861938-0700 Tokens available: -7069
2024-06-22T17:17:18.865433-0700 Temperature: 0.5907771934426157


Initializing Autonomous Agent Boss Agent Creator...
Agent generating function schema

Loop 1 of 1




To create and execute a swarm of agents for automating customer support for an e-commerce platform, we will use the `create_and_execute_swarm` function. This function will create a list of specialized LLM agents, each with a specific role in customer support. Here's the breakdown of the task: * **Function Name**: `create_and_execute_swarm` * **Arguments**: + `name`: A list of names for the agents. + `system_prompt`: A list of system prompts for the agents. + `task`: The description of the task for the swarm. For the e-commerce platform, we can specify the names and system prompts for each agent. For instance, the agents can be named 'Product Inquiry Agent', 'Order Status Agent', 'Complaint Resolution Agent', etc. Each agent will have a system prompt tailored to its role. Let's proceed with creating and executing this swarm. The JSON output will adhere to the schema provided for the `cr

2024-06-22T17:17:28.072930-0700 Attempt 1: Error generating response: expected string or bytes-like object, got 'NoneType'


{ "type": "function", "function": { "name": "create_and_execute_swarm", "parameters": { "name": [ "CS_Agent1", "CS_Agent2", "CS_Agent3" ], "system_prompt": [ "Greet customer and identify issue", "Provide solutions or escalate issue", "Follow up and ensure resolution" ], "task": "Automating customer support for an e-commerce platform" } } 

2024-06-22T17:17:30.692259-0700 Attempt 2: Error generating response: expected string or bytes-like object, got 'NoneType'



{ "type": "function", "function": { "name": "create_and_execute_swarm", "parameters": { "name": [ "CustomerSupportAgent1", "CustomerSupportAgent2", "CustomerSupportAgent3" ], "system_prompt": [ "Assist customers with product inquiries", "Resolve customer complaints and issues", "Provide information on order status and shipping" ], "task": "Automate customer support for an e-commerce platform" } } 

2024-06-22T17:17:33.539722-0700 Attempt 3: Error generating response: expected string or bytes-like object, got 'NoneType'
2024-06-22T17:17:33.540472-0700 Failed to generate a valid response after retry attempts.
2024-06-22T17:17:33.540701-0700 Autosaving agent state.
2024-06-22T17:17:33.540904-0700 Saving Agent Boss Agent Creator state to: Boss Agent Creator_state.json



Error saving agent state: Object of type TikTokenizer is not JSON serializable


In [43]:
from swarms.utils.loguru_logger import logger


def run_jamba_swarm(task: str = None):
    logger.info(f"Making plan for the task: {task}")
    out = planning_agent.run(task)
    
    memory = planning_agent.short_memory.return_history_as_string()

    # Boss agent
    return boss_agent_creator.run(memory)
    
    
# Example usage
run_jamba_swarm("Create a swarm of agents for automating customer support for an e-commerce platform.")


2024-06-22T17:11:19.727696-0700 Making plan for the task: Create a swarm of agents for automating customer support for an e-commerce platform.
2024-06-22T17:11:19.738286-0700 Autonomous Agent Activated.
2024-06-22T17:11:19.739463-0700 All systems operational. Executing task...
2024-06-22T17:11:19.758047-0700 Tokens available: -7261
2024-06-22T17:11:19.759725-0700 Temperature: 0.31327886639849234



Loop 1 of 1




To automate customer support for an e-commerce platform using a swarm of specialized LLM agents, we will follow the outlined structure. Here's the detailed plan: ### 1. **Business Problem Intake:** * **Objective:** Automate customer support to efficiently handle inquiries, complaints, and provide information. * **Constraints:** Real-time response capability, ability to handle a high volume of queries, and integration with the e-commerce platform. * **Expected Outcomes:** Improved customer satisfaction, reduced workload on human customer support agents, and efficient handling of a wide range of customer inquiries. * **Key Components:** Query classification, order status updates, product recommendations, and technical support. ### 2. **Agent Design:** * **Number of Agents:** 4 specialized agents. * **Agent Roles and Tools:** 1. **Query Classifier Agent:** Classifies queries into categories like order status, product inquiries, technical issues, etc. + Tools: Natural lang

2024-06-22T17:11:31.672062-0700 Autosaving agent state.
2024-06-22T17:11:31.673920-0700 Saving Agent Boss Director state to: Boss Director_state.json


Error saving agent state: Object of type TikTokenizer is not JSON serializable


2024-06-22T17:11:31.676071-0700 Autonomous Agent Activated.
2024-06-22T17:11:31.676582-0700 All systems operational. Executing task...
2024-06-22T17:11:31.681411-0700 Tokens available: -5216
2024-06-22T17:11:31.682256-0700 Temperature: 0.46020517238764247


...


{ "type": "function", "function": { "name": "create_and_execute_swarm", "parameters": { "name": ["Customer Query Classifier Agent", "Order Status Agent", "Product Recommendation Agent", "Technical Support Agent"], "system_prompt": ["Classify incoming customer queries into predefined categories.", "Provide order status updates to customers.", "Suggest products to customers based on their query and browsing history.", "Assist customers with technical issues."], "task": "Automate customer support for an e-commerce platform." } } } 

2024-06-22T17:11:40.292901-0700 Attempt 2: Error generating response: expected string or bytes-like object, got 'NoneType'



{ "type": "function", "function": { "name": "create_and_execute_swarm", "parameters": { "name": [ "Customer Query Classifier Agent", "Order Status Agent", "Product Recommendation Agent", "Technical Support Agent" ], "system_prompt": [ "Classify incoming customer queries into predefined categories.", "Provide order status updates to customers.", "Suggest products to customers based on their query and browsing history.", "Assist customers with technical issues." ], "task": "Automate customer support for an e-commerce platform." } } 

2024-06-22T17:11:43.262321-0700 Attempt 3: Error generating response: expected string or bytes-like object, got 'NoneType'
2024-06-22T17:11:43.263663-0700 Failed to generate a valid response after retry attempts.
2024-06-22T17:11:43.264713-0700 Autosaving agent state.
2024-06-22T17:11:43.265105-0700 Saving Agent Boss Agent Creator state to: Boss Agent Creator_state.json



Error saving agent state: Object of type TikTokenizer is not JSON serializable


In [ ]:
import time

from pydantic import BaseModel


class AgentSchema(BaseModel):
    name: str = None
    system_prompt: str = None
    task: str = None
    response: str = None

class JambaSwarmRequest(BaseModel):
    task: str = None,
    plan: str = None
    agents: List[AgentSchema] = None
    timestamp: int = int(time.time())
    
class JambaSwarmResponse(BaseModel):
    task: str = None,
    plan: str = None
    agents: List[AgentSchema] = None
    timestamp: int = int(time.time())
    response: str = None